In [1]:
!git clone https://github.com/mantasbandonis/denoiser

Cloning into 'denoiser'...
remote: Enumerating objects: 16639, done.
remote: Counting objects: 100% (3207/3207), done.
remote: Compressing objects: 100% (3205/3205), done.
remote: Total 16639 (delta 2), reused 3206 (delta 1), pack-reused 13432
Receiving objects: 100% (16639/16639), 343.05 MiB | 33.96 MiB/s, done.
Resolving deltas: 100% (187/187), done.
Checking out files: 100% (8437/8437), done.


In [3]:
import os
os.getcwd()

'/content'

In [4]:
%cd /content/denoiser

/content/denoiser


In [ ]:
#%cd /content/denoiser/dataset/debug

/content/denoiser/dataset/debug


In [4]:
!pip install -r requirements.txt

  Cloning https://github.com/ludlows/python-pesq to /tmp/pip-install-mj5_1rrj/pesq_3ba0020da5a9436ea59b7446104e729a
  Running command git clone -q https://github.com/ludlows/python-pesq /tmp/pip-install-mj5_1rrj/pesq_3ba0020da5a9436ea59b7446104e729a
     |████████████████████████████████| 72 kB 626 kB/s 
     |████████████████████████████████| 58 kB 8.6 MB/s 
     |████████████████████████████████| 14.6 MB 38.1 MB/s 
     |████████████████████████████████| 3.2 MB 54.0 MB/s 
     |████████████████████████████████| 753.2 MB 13 kB/s 
     |████████████████████████████████| 58 kB 8.2 MB/s 
     |████████████████████████████████| 54 kB 3.1 MB/s 
     |████████████████████████████████| 41 kB 670 kB/s 
     |████████████████████████████████| 41 kB 192 kB/s 
     |████████████████████████████████| 40 kB 7.2 MB/s 
     |████████████████████████████████| 40 kB 6.1 MB/s 
  Created wheel for pesq: filename=pesq-0.0.3-cp37-cp37m-linux_x86_64.whl size=211379 sha256=b6ffdcffc89b7e8f9cdacd85c43256e5e2

In [5]:
!pip install denoiser

     |████████████████████████████████| 48 kB 3.9 MB/s 
  Created wheel for denoiser: filename=denoiser-0.1.4-py3-none-any.whl size=42782 sha256=7574da5c58137064aa40a95d31675d5c51fb28734a3530bf653e2da80d8bc68a
  Stored in directory: /root/.cache/pip/wheels/76/d9/3d/f34ed4201f8090d6c2adf117ba05acec0c4635d0ed7416e0d9
Successfully built denoiser


In [6]:
!python -m denoiser.audio

[]

In [5]:
%mkdir egs/debug/tr
!python -m denoiser.audio dataset/debug/noisy > egs/debug/tr/noisy.json
!python -m denoiser.audio dataset/debug/clean > egs/debug/tr/clean.json

In [6]:
!pip install --upgrade numpy

     |████████████████████████████████| 15.7 MB 9.0 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.0
    Uninstalling numpy-1.19.0:
      Successfully uninstalled numpy-1.19.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.5.1 which is incompatible.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.5.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [8]:
!pip install pydub

In [57]:
from pydub import AudioSegment as am
import glob
import librosa
import soundfile as sf

In [58]:
path = "dataset/debug/clean"

In [ ]:
for file in sorted(glob.glob(os.path.join(path, '*.wav'))):
    x, sr = librosa.load(file, sr=11050)
    y = librosa.resample(x, 11050, 8000)
    sf.write(file, y, 8000)

In [40]:
path = "dataset/debug/noisy"

In [49]:
for file in sorted(glob.glob(os.path.join(path, '*.wav'))):
    x, sr = librosa.load(file, sr=11050)
    y = librosa.resample(x, 11050, 8000)
    sf.write(file, y, 8000)

In [50]:
!./train.py

[2022-04-30 15:15:14,092][__main__][INFO] - For logs, checkpoints and samples check /content/denoiser/outputs/exp_
[2022-04-30 15:15:16,901][denoiser.solver][INFO] - Loading checkpoint model: checkpoint.th
[2022-04-30 15:15:17,224][denoiser.solver][INFO] - Replaying metrics from previous run
[2022-04-30 15:15:17,225][denoiser.solver][INFO] - Epoch 1: Train=0.01255 Valid=0.03731 Best=0.03731
[2022-04-30 15:15:17,225][denoiser.solver][INFO] - ----------------------------------------------------------------------
[2022-04-30 15:15:17,225][denoiser.solver][INFO] - Training...
[2022-04-30 15:15:23,079][denoiser.solver][INFO] - Train | Epoch 2 | 8/41 | 1.6 it/sec | Loss 0.01349


In [43]:
import wave
import audioop

In [ ]:
def downsampleWav(src, dst, inrate=11025, outrate=16000, inchannels=2, outchannels=1):
    if not os.path.exists(src):
        print('Source not found!')
        return False

    if not os.path.exists(os.path.dirname(dst)):
        os.makedirs(os.path.dirname(dst))

    try:
        s_read = wave.open(src, 'rb')
        s_write = wave.open(dst, 'wb')
    except:
        print('Failed to open files!')
        return False

    n_frames = s_read.getnframes()
    data = s_read.readframes(n_frames)

    try:
        converted = audioop.ratecv(data, 2, inchannels, inrate, outrate, None)
        if outchannels == 1:
            converted = audioop.tomono(converted[0], 2, 1, 0)
    except:
        print('Failed to downsample wav')
        return False

    try:
        s_write.setparams((outchannels, 2, outrate, 0, 'NONE', 'Uncompressed'))
        s_write.writeframes(converted)
    except:
        print('Failed to write wav')
        return False

    try:
        s_read.close()
        s_write.close()
    except:
        print('Failed to close wav files')
        return False

    return True

In [ ]:
os.path.exists("/content/denoiser/dataset/debug/clean")

True

In [ ]:
src = "/content/denoiser/dataset/debug/clean"
dst = "/content/denoiser/dataset/debug/clean"

In [ ]:
inrate=11025
outrate=8000
inchannels=2
outchannels=0

In [ ]:
import librosa

In [ ]:
for filename in glob.glob(os.path.join(path, '*.wav')):  
  #samples, sample_rate = librosa.load(filename, sr = 16000)
  s_read = wave.open(filename, 'rb')
  s_write = wave.open(filename, 'wb')
  n_frames = s_read.getnframes()
  data = s_read.readframes(n_frames)

  samples, sample_rate = librosa.load(data, sr = 16000)

In [ ]:
import wave, os, glob
path = '/content/denoiser/dataset/debug/clean'


for filename in glob.glob(os.path.join(path, '*.wav')):  
    try:
        s_read = wave.open(filename, 'rb')
        s_write = wave.open(filename, 'wb')
    except:
        print('Failed to open files!')

    n_frames = s_read.getnframes()
    data = s_read.readframes(n_frames)

    try:
        converted = audioop.ratecv(data, 2, inchannels, inrate, outrate, None)
        if outchannels == 1:
            converted = audioop.tomono(converted[0], 2, 1, 0)
    except:
        print('Failed to downsample wav')

    try:
        #s_write.setparams((outchannels, 2, outrate, 0, 'NONE', 'Uncompressed'))
        s_write.writeframes(converted)
    except:
        print('Failed to write wav')

    try:
        s_read.close()
        s_write.close()
    except:
        print('Failed to close wav files')

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
Failed to write wav
Failed to close wav files
Failed to downsample wav
Failed to write wav
Failed to close wav files
Failed to downsample wav
Failed to write wav
Failed to close wav files
Failed to downsample wav
Failed to write wav
Failed to close wav files
Failed to downsample wav
Failed to write wav
Failed to close wav files
Failed to downsample wav
Failed to write wav
Failed to close wav files
Failed to downsample wav
Failed to write wav
Failed to close wav files
Failed to downsample wav
Failed to write wav
Failed to close wav files
Failed to downsample wav
Failed to write wav
Failed to close wav files
Failed to downsample wav
Failed to write wav
Failed to close wav files
Failed to downsample wav
Failed to write wav
Failed to close wav files
Failed to downsample wav
Failed to write wav
Failed to close wav files
Failed to downsample wav
Failed to write wav
Failed to close wav files
Failed to downsample wav
Failed to 

In [ ]:
for fil in src:
  downsampleWav(fil, fil, inrate=11025, outrate=16000, inchannels=2, outchannels=1)

Failed to open files!
Source not found!
Source not found!
Source not found!
Source not found!
Source not found!
Source not found!
Source not found!
Failed to open files!
Source not found!
Source not found!
Source not found!
Source not found!
Source not found!
Source not found!
Source not found!
Source not found!
Failed to open files!
Source not found!
Source not found!
Source not found!
Source not found!
Source not found!
Source not found!
Source not found!
Failed to open files!
Source not found!
Source not found!
Source not found!
Source not found!
Source not found!
Failed to open files!
Source not found!
Source not found!
Source not found!
Source not found!
Source not found!


In [51]:
!./train.py continue_pretrained=dns64 demucs.hidden=64

[2022-04-30 15:15:40,158][__main__][INFO] - For logs, checkpoints and samples check /content/denoiser/outputs/exp_continue_pretrained=dns64,demucs.hidden=64
[2022-04-30 15:15:42,979][denoiser.solver][INFO] - Fine tuning from pre-trained model dns64
Downloading: "https://dl.fbaipublicfiles.com/adiyoss/denoiser/dns64-a7761ff99a7d5bb6.th" to /root/.cache/torch/checkpoints/dns64-a7761ff99a7d5bb6.th
100% 128M/128M [00:01<00:00, 76.4MB/s]
[2022-04-30 15:15:45,401][denoiser.solver][INFO] - ----------------------------------------------------------------------
[2022-04-30 15:15:45,401][denoiser.solver][INFO] - Training...
[2022-04-30 15:15:52,504][denoiser.solver][INFO] - Train | Epoch 1 | 8/41 | 1.3 it/sec | Loss 0.01756
[2022-04-30 15:15:59,267][denoiser.solver][INFO] - Train | Epoch 1 | 16/41 | 1.2 it/sec | Loss 0.01572
[2022-04-30 15:16:06,024][denoiser.solver][INFO] - Train | Epoch 1 | 24/41 | 1.2 it/sec | Loss 0.01435
[2022-04-30 15:16:12,783][denoiser.solver][INFO] - Train | Epoch 1 | 3

In [52]:
!python -m denoiser.evaluate --model_path="/content/denoiser/outputs/exp_/best.th" --data_dir "egs/debug/tr" --no

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
       

       Run model on reference ref and degraded deg
       Sample rate (fs) - No default. Must select either 8000 or 16000.
       Note there is narrow band (nb) mode only when sampling rate is 8000Hz.
       

       Run model on reference ref and degraded deg
       Sample rate (fs) - No default. Must select either 8000 or 16000.
       Note there is narrow band (nb) mode only when sampling rate is 8000Hz.
       

       Run model on reference ref and degraded deg
       Sample rate (fs) - No default. Must select either 8000 or 16000.
       Note there is narrow band (nb) mode only when sampling rate is 8000Hz.
       

       Run model on reference ref and degraded deg
       Sample rate (fs) - No default. Must select either 8000 or 16000.
       Note there is narrow band (nb) mode only when sampling rate is 8000Hz.
       

       Run model on reference ref and degraded deg
       Sample rate (fs) - No defa

In [ ]:
!python -m denoiser.enhance --model_path="/content/denoiser/outputs/exp_continue_pretrained=dns64,demucs.hidden=64/best.th"  --num_workers=4  --streaming --out_dir="/content/denoiser/dataset/debug/out_test" --noisy_dir="/content/denoiser/dataset/debug/noisy_test"

INFO:denoiser.pretrained:Loading model from /content/denoiser/outputs/exp_continue_pretrained=dns64,demucs.hidden=64/best.th
INFO:__main__:Generate enhanced files | 639/3195 | 885.7 it/sec
INFO:__main__:Generate enhanced files | 1278/3195 | 835.4 it/sec
INFO:__main__:Generate enhanced files | 1917/3195 | 706.3 it/sec
INFO:__main__:Generate enhanced files | 2556/3195 | 691.1 it/sec
INFO:__main__:Generate enhanced files | 3195/3195 | 626.2 it/sec
Waiting for pending jobs...
INFO:__main__:Generate enhanced files | 639/3195 | 2.3 it/sec
INFO:__main__:Generate enhanced files | 1278/3195 | 2.3 it/sec
INFO:__main__:Generate enhanced files | 1917/3195 | 2.3 it/sec
INFO:__main__:Generate enhanced files | 2556/3195 | 2.3 it/sec
INFO:__main__:Generate enhanced files | 3195/3195 | 2.3 it/sec


In [ ]:
 !zip -r out_test.zip /content/denoiser/dataset/debug/out_test 

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
updating: content/denoiser/dataset/debug/out_test/n2942_noisy.wav (deflated 6%)
updating: content/denoiser/dataset/debug/out_test/n0939_noisy.wav (deflated 5%)
updating: content/denoiser/dataset/debug/out_test/n0551_enhanced.wav (deflated 12%)
updating: content/denoiser/dataset/debug/out_test/n1861_noisy.wav (deflated 5%)
updating: content/denoiser/dataset/debug/out_test/n1570_noisy.wav (deflated 5%)
updating: content/denoiser/dataset/debug/out_test/n2731_noisy.wav (deflated 5%)
updating: content/denoiser/dataset/debug/out_test/n1694_enhanced.wav (deflated 22%)
updating: content/denoiser/dataset/debug/out_test/n0688_enhanced.wav (deflated 18%)
updating: content/denoiser/dataset/debug/out_test/n2343_enhanced.wav (deflated 18%)
updating: content/denoiser/dataset/debug/out_test/n1064_noisy.wav (deflated 6%)
updating: content/denoiser/dataset/debug/out_test/n2246_noisy.wav (deflated 3%)
updating: content/denoiser/dataset/de

In [ ]:
!git config --global user.email "mantas.bandonis@outlook.com"

In [ ]:
!git config --global user.name "mantasbandonis"

In [ ]:
!git add . && git commit -m "commit changes"

[main 7f303572] commit changes
 4858 files changed, 0 insertions(+), 0 deletions(-)
 rewrite dataset/debug/out_test/n0001_enhanced.wav (99%)
 rewrite dataset/debug/out_test/n0002_enhanced.wav (99%)
 rewrite dataset/debug/out_test/n0003_enhanced.wav (98%)
 rewrite dataset/debug/out_test/n0004_enhanced.wav (99%)
 rewrite dataset/debug/out_test/n0005_enhanced.wav (99%)
 rewrite dataset/debug/out_test/n0006_enhanced.wav (99%)
 rewrite dataset/debug/out_test/n0007_enhanced.wav (99%)
 rewrite dataset/debug/out_test/n0008_enhanced.wav (99%)
 rewrite dataset/debug/out_test/n0009_enhanced.wav (99%)
 rewrite dataset/debug/out_test/n0010_enhanced.wav (99%)
 rewrite dataset/debug/out_test/n0011_enhanced.wav (99%)
 rewrite dataset/debug/out_test/n0012_enhanced.wav (99%)
 rewrite dataset/debug/out_test/n0013_enhanced.wav (99%)
 rewrite dataset/debug/out_test/n0014_enhanced.wav (99%)
 rewrite dataset/debug/out_test/n0015_enhanced.wav (99%)
 rewrite dataset/debug/out_test/n0016_enhanced.wav (99%)
 rew

In [ ]:
!git remote set-url origin https://github.com/mantasbandonis/denoiser.git

In [ ]:
 !git push origin --all

fatal: could not read Username for 'https://github.com': No such device or address
